<a href="https://colab.research.google.com/github/ecloguehwang/HSS/blob/master/%EC%9A%B0%EC%9E%A5%EC%82%B0%ED%9E%90%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%8A%B8%EC%95%84%ED%8C%8C%ED%8A%B8_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#colab에서 matplotlib와 sns 라이브러리 그래프 한글을 깨지지 않게 하는 법A: 이 코드실행하고 런타임(runtime) 다시 실행하기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
!pip install selenium

#동적html제어:
전체거래방식 클릭-매매 클릭-전체면적 클릭-109클릭-112클릭-가격순 클릭

In [38]:
#코드 제작중: 0213v - 멋진 코드!
##동적html제어: 전체거래방식 클릭-매매 클릭-전체면적 클릭-109클릭-112클릭-동일매물 묶음 - 가격순 클릭


import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pytz

def get_data():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)
    driver.get('https://new.land.naver.com/complexes/11337?ms=37.556255,126.840704,17&a=APT:ABYG:JGC:PRE&e=RETAIL')
    time.sleep(5)

    wait = WebDriverWait(driver, 10)

    # 거래 방식 클릭 (전체 -> 매매)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[1]/button/span'))).click()
    time.sleep(1)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[1]/div/div/ul/li[2]/label'))).click()
    time.sleep(2)


    # 면적 선택 (전체 -> 109㎡, 112㎡)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[2]/button/span'))).click()
    time.sleep(1)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[2]/div/div/ul/li[3]/label'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[2]/div/div/ul/li[4]/label'))).click()
    time.sleep(2)


    # 면적 선택 닫기 버튼 클릭 (추가된 부분)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[2]/div/div[2]/div/button'))).click()
    time.sleep(1)

    #동일매물 묶기
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="complexOverviewList"]/div[2]/div[1]/div[3]/div/label'))).click()
    time.sleep(1)


    # 가격순 정렬
    #wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="articleListArea"]/div[1]/div[2]/button[2]'))).click()
    #time.sleep(3)


    # 필터 적용 후 데이터 로드 대기
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'item_inner')))

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    containers = soup.find_all('div', class_='item_inner')

    apartment_info = []
    transaction_types = []
    prices = []
    property_specs = []
    real_estate_offices = []
    confirm_date = []

    for container in containers:

        #동호수
        info_span = container.find('span', class_='text')
        apartment_info.append(info_span.get_text(strip=True) if info_span else '')

        #매매
        type_span = container.find('span', class_='type')
        transaction_types.append(type_span.get_text(strip=True) if type_span else '')

        #가격
        price_span = container.find('span', class_='price')
        prices.append(price_span.get_text(strip=True) if price_span else '')

        #물건설명
        specs = container.find_all('span', class_='spec')
        specs_text = [spec.get_text(strip=True) for spec in specs]
        property_specs.append(' / '.join(specs_text))

        #부동산
        agent_anchor = container.find('a', class_='agent_name', attrs={'data-nclk': lambda x: x and 'TAA.realtor' in x})
        real_estate_offices.append(agent_anchor.get_text(strip=True) if agent_anchor else '')


        #물건확인날짜
        confirm_label = container.find('em', class_='data')
        confirm_date.append(confirm_label.get_text(strip=True) if confirm_label else '')



    driver.quit()

    return apartment_info, transaction_types, prices, property_specs, real_estate_offices, confirm_date

apartment_info, transaction_types, prices, property_specs, real_estate_offices, confirm_date = get_data()

data_list = list(zip(apartment_info, transaction_types, prices, property_specs, real_estate_offices, confirm_date))
df = pd.DataFrame(data_list, columns=['동호수', '거래 방식', '가격', '물건 설명', '부동산', '확인날짜'])


kst = pytz.timezone('Asia/Seoul')
current_datetime = datetime.now(kst).strftime('%Y%m%d_%H%M')
file_name = f'hillstate_{current_datetime}.xlsx'
path = f'/content/drive/MyDrive/python/crawling/{file_name}'

df.to_excel(path, index=False)

print(f"데이터가 '{path}' 경로에 엑셀 파일로 저장되었습니다!")
print("수집된 아파트 정보:", apartment_info)

데이터가 '/content/drive/MyDrive/python/crawling/hillstate_20250213_2124.xlsx' 경로에 엑셀 파일로 저장되었습니다!
수집된 아파트 정보: ['우장산힐스테이트 134동', '', '우장산힐스테이트 112동', '우장산힐스테이트 133동', '우장산힐스테이트 108동', '', '우장산힐스테이트 134동', '우장산힐스테이트 113동', '', '우장산힐스테이트 105동', '우장산힐스테이트 134동', '우장산힐스테이트 102동', '', '우장산힐스테이트 112동', '', '우장산힐스테이트 113동', '우장산힐스테이트 117동', '', '우장산힐스테이트 137동', '우장산힐스테이트 140동', '', '우장산힐스테이트 134동', '우장산힐스테이트 115동', '우장산힐스테이트 115동', '우장산힐스테이트 117동', '우장산힐스테이트 115동', '우장산힐스테이트 132동']


#힐스테이트 매물 : 첫번째 페이지의 20개 추출

In [22]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pytz


def get_data():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)

    base_url = 'https://new.land.naver.com/complexes/11337?ms=37.556255,126.840704,17&a=APT:ABYG:JGC:PRE&e=RETAIL'
    driver.get(base_url)
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    containers = soup.find_all('div', class_='item_inner')

    apartment_info = []
    transaction_types = []
    prices = []
    property_specs = []
    real_estate_offices = []
    confirm_date = []

    for container in containers:
        info_span = container.find('span', class_='text')
        apartment_info.append(info_span.get_text(strip=True) if info_span else '')

        type_span = container.find('span', class_='type')
        transaction_types.append(type_span.get_text(strip=True) if type_span else '')

        price_span = container.find('span', class_='price')
        prices.append(price_span.get_text(strip=True) if price_span else '')

        specs = container.find_all('span', class_='spec')
        specs_text = [spec.get_text(strip=True) for spec in specs]
        property_specs.append(' / '.join(specs_text))

        agent_anchor = container.find('a', class_='agent_name', attrs={'data-nclk': lambda x: x and 'TAA.realtor' in x})
        real_estate_offices.append(agent_anchor.get_text(strip=True) if agent_anchor else '')


        #물건확인날짜
        confirm_label = container.find('em', class_='data')
        confirm_date.append(confirm_label.get_text(strip=True) if confirm_label else '')

    driver.quit()

    return apartment_info, transaction_types, prices, property_specs, real_estate_offices, confirm_date

apartment_info, transaction_types, prices, property_specs, real_estate_offices, confirm_date = get_data()

data_list = list(zip(apartment_info, transaction_types, prices, property_specs, real_estate_offices, confirm_date))
df = pd.DataFrame(data_list, columns=['동호수', '거래 방식', '가격', '물건 설명', '부동산', '확인날짜'])


kst = pytz.timezone('Asia/Seoul')
current_datetime = datetime.now(kst).strftime('%Y%m%d_%H%M')
file_name = f'hillstate_{current_datetime}.xlsx'
path = f'/content/drive/MyDrive/python/crawling/{file_name}'

df.to_excel(path, index=False)

print(f"데이터가 '{path}' 경로에 엑셀 파일로 저장되었습니다!")

데이터가 '/content/drive/MyDrive/python/crawling/hillstate_20250213_1757.xlsx' 경로에 엑셀 파일로 저장되었습니다!


#매물번호 리스트에 넣어놓으면 32평 물건 찾는 코드

In [ ]:

import pandas as pd
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import time

def get_data(article_no):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)

    base_url = 'https://new.land.naver.com/complexes/11337?ms=37.556255,126.840704,17&a=APT:ABYG:JGC:PRE&e=RETAIL&ad=true&articleNo='
    url = f"{base_url}{article_no}"

    driver.get(url)
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    phone_numbers = [dd.get_text() for dd in soup.find_all('dd', class_='text text--number')]
    real_estate_offices = [strong.get_text() for strong in soup.find_all('strong', class_='info_title')]

    apartment_info = []
    for h4 in soup.find_all('h4', class_='info_title'):
        title_name = h4.find('strong', class_='info_title_name').get_text() if h4.find('strong', class_='info_title_name') else ''
        floor_info = h4.get_text(strip=True).replace(title_name, '')
        apartment_info.append(f"{title_name} {floor_info}")

    driver.quit()

    return phone_numbers, real_estate_offices, apartment_info

def extract_price(apartment_str):
    """ '매매' 키워드 오른쪽 문자열을 숫자형으로 변환 """
    match = re.search(r'매매\s*([\d억천만, ]+)', apartment_str)
    if match:
        price_str = match.group(1)
        return convert_price_to_int(price_str)
    return None

def convert_price_to_int(price_str):
    price_str = price_str.replace(',', '').replace(' ', '').strip()
    total_price = 0

    # '억' 단위 처리
    if '억' in price_str:
        parts = re.split(r'억', price_str)
        if parts[0]:
            total_price += int(parts[0]) * 10**8
        price_str = parts[1] if len(parts) > 1 else ''

    # '천' 단위 처리 (천만원)
    if '천' in price_str:
        parts = re.split(r'천', price_str)
        if parts[0]:
            total_price += int(parts[0]) * 10**7
        price_str = parts[1] if len(parts) > 1 else ''

    # '만' 단위 처리
    if '만' in price_str:
        parts = re.split(r'만', price_str)
        if parts[0]:
            total_price += int(parts[0]) * 10**4
        price_str = parts[1] if len(parts) > 1 else ''

    # 남은 숫자 처리 (만원 단위로 가정)
    if price_str and price_str.isdigit():
        total_price += int(price_str) * 10**4

    return total_price if total_price != 0 else None

article_numbers = list(set([
    2504822211, 2504210136, 2505908173, 2505703456, 2503752598, 2501572048, 2502735210, 2502294399,
    2501662043, 2502293027, 2502356205, 2502814467, 2505703905, 2501607928, 2501658024, 2505050574,
    2501996356, 2504771773, 2502179793, 2504023203, 2503791928, 2502618284, 2505796375, 2503086595,
    2504444743, 2503516163, 2504694825, 2502357964, 2502025913, 2502353435, 2503170922, 2506080772
]))

data_list = []
serial_number = 1

for article_no in article_numbers:
    phone_numbers, real_estate_offices, apartment_info = get_data(article_no)
    apartment_str = ' '.join(apartment_info)  # 아파트 정보 통합
    price = extract_price(apartment_str)

    # 중복된 매물번호 체크
    existing_articles = [row[1] for row in data_list]
    if article_no not in existing_articles:
        data_list.append([
            serial_number,
            article_no,
            apartment_str,
            price,
            real_estate_offices,
            phone_numbers
        ])
        serial_number += 1

# 엑셀 저장 경로 설정
path = '/content/drive/MyDrive/python/crawling/hillstate.xlsx'
df = pd.DataFrame(data_list, columns=['일련번호', '매물번호', '아파트 동 호수', '가격', '부동산', '전화번호'])

# 엑셀 파일 저장
df.to_excel(path, index=False)

# 결과 출력
print(f"데이터가 '{path}' 경로에 엑셀 파일로 저장되었습니다!")
print("\n===== 수집된 데이터 =====\n")
print(df.head(10))

데이터가 '/content/drive/MyDrive/python/crawling/hillstate.xlsx' 경로에 엑셀 파일로 저장되었습니다!

===== 수집된 데이터 =====

   일련번호        매물번호                                        아파트 동 호수  \
0     1  2502814467  우장산힐스테이트 115동 13층매매13억 7,000 우장산힐스테이트 115동 13층   
1     2  2503086595          우장산힐스테이트 140동 고층매매15억 우장산힐스테이트 140동 고층   
2     3  2503516163        우장산힐스테이트 115동 16층매매15억 우장산힐스테이트 115동 16층   
3     4  2506080772    우장산힐스테이트 134동 3층매매13억 5,000 우장산힐스테이트 134동 3층   
4     5  2503752598    우장산힐스테이트 117동 중층매매13억 4,000 우장산힐스테이트 117동 중층   
5     6  2505796375          우장산힐스테이트 140동 고층매매15억 우장산힐스테이트 140동 고층   
6     7  2502353435        우장산힐스테이트 102동 12층매매15억 우장산힐스테이트 102동 12층   
7     8  2505703456    우장산힐스테이트 117동 중층매매13억 4,000 우장산힐스테이트 117동 중층   
8     9  2502293027    우장산힐스테이트 134동 저층매매13억 5,000 우장산힐스테이트 134동 저층   
9    10  2504023203                                                   

             가격                   부동산  \
0  1.370000e+09       [샘터부동산공인중개사사무소]   
1  1.500000e+09       [부동산뉴스공인중개사

#매물번호 입력하면 32평 보여주는 코드: chatGPT legacy모델

In [ ]:
#처음에 chatGPT legacy모델로 만든 코드

from selenium import webdriver
from bs4 import BeautifulSoup
import time

def get_data(article_no):
    # 크롬 드라이버 설정 (드라이버 경로 수정 필요)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 브라우저 창 없이 실행
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=options)

    # 유동적인 URL 설정
    base_url = 'https://new.land.naver.com/complexes/11337?ms=37.556255,126.840704,17&a=APT:ABYG:JGC:PRE&e=RETAIL&ad=true&articleNo='

    url = f"{base_url}{article_no}"

    driver.get(url)

    # 페이지 로딩 대기
    time.sleep(5)

    # 페이지 소스 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 전화번호 크롤링
    phone_numbers = [dd.get_text() for dd in soup.find_all('dd', class_='text text--number')]

    # 부동산 사무소 이름 크롤링
    real_estate_offices = [strong.get_text() for strong in soup.find_all('strong', class_='info_title')]

    # 아파트 동 및 호수 정보 추출
    apartment_info = []
    for h4 in soup.find_all('h4', class_='info_title'):
        title_name = h4.find('strong', class_='info_title_name').get_text() if h4.find('strong', class_='info_title_name') else ''
        floor_info = h4.get_text(strip=True).replace(title_name, '')  # 동 정보 제거 후 층 정보만 남기기
        apartment_info.append(f"{title_name} {floor_info}")

    driver.quit()

    return phone_numbers, real_estate_offices, apartment_info


# 주어진 article 번호 리스트
article_numbers = [
    2504822211, 2504210136, 2505908173, 2505703456, 2503752598, 2501572048, 2502735210, 2502294399,
    2501662043, 2502293027, 2502356205, 2502814467, 2505703905, 2501607928, 2501658024, 2505050574,
    2501996356, 2504771773, 2502179793, 2504023203, 2503791928, 2502618284, 2505796375, 2503086595,
    2504444743, 2503516163, 2504694825, 2502357964, 2502025913, 2502353435, 2503170922
]

# 결과 출력
for article_no in article_numbers:
    phone_numbers, real_estate_offices, apartment_info = get_data(article_no)
    print(f"매물번호: {article_no}")
    print("아파트 동 호수:", apartment_info if apartment_info else "정보 없음")
    print("전화번호:", phone_numbers)
    print("부동산:", real_estate_offices)
    print('-' * 50)

In [9]:
#전처리한 파일 불러오기
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

#matplotlib에서 한글구현
plt.rc('font', family='NanumBarunGothic')

#그래프 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

path = '/content/drive/MyDrive/python/crawling/hillstate_20250213_1631.xlsx'
df = pd.read_excel(path)
df.head()

,동호수,거래 방식,가격,물건 설명,부동산,확인날짜
0,우장산힐스테이트 134동,매매,"13억 7,000","109/84m², 4/21층, 남동향 / 올확장 주인거주 학군우수 우장산뷰 발산역세...",골드공인중개사사무소,25.02.13.
1,우장산힐스테이트 127동,전세,10억,"158/126m², 8/17층, 남향 / 로얄동 로얄층 남향 거실 방확장 올리모델링...",금돼지부동산공인중개사사무소,25.02.13.
2,우장산힐스테이트 116동,전세,"8억 2,000","132/101m², 3/17층, 남향 / 발산역 역세권 단지중앙 남향 해잘드는동 확...",금돼지부동산공인중개사사무소,25.02.13.
3,우장산힐스테이트 138동,전세,"7억 5,000","112/84m², 19/20층, 남서향 / 올수리.환하고 예쁜집.빠른입주가.햇빛 짱짱한집",샘터부동산공인중개사사무소,25.02.13.
4,우장산힐스테이트 138동,전세,"7억 5,000","112/84m², 19/20층, 남서향 / 강추역세권우수학군 우장산굿 남서향 아주...",금돼지부동산공인중개사사무소,25.02.13.


In [17]:
#부동산별 빈도
df_freq = df.groupby('부동산').size().sort_values(ascending=False)
df_freq

,0
부동산,
샘터부동산공인중개사사무소,4
금돼지부동산공인중개사사무소,3
일등힐스테이트부동산공인중개사사무소,3
까치부동산중개사무소,2
명문효자부동산공인중개사사무소,2
힐스타부동산공인중개사사무소,2
골드공인중개사사무소,1
대신공인중개사,1
마곡목화부동산공인중개사사무소,1
